In [1]:
import ollama
import random
import time
import json

In [2]:

# Hilfsfunktionen
def load_list_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file if line.strip()]

In [3]:
def generate_date(raw=False):
    monat = random.choice(monat_list)
    jahr = random.choice(jahr_list)
    return (monat, jahr) if raw else f"im {monat} {jahr}"

In [4]:
def find_span(text, target):
    start = text.find(target)
    return (start, start + len(target)) if start != -1 else None

In [37]:
def build_prompt_type1(name, ort, verb_sample, datum_str):
    return (
        f"Erstelle einen deutschen Satz über die Person '{name}', "
        f"die sich in '{ort}' aufhält oder dort etwas tut. "
        f"Verwende exakt diesen Namen: {name}, und exakt diesen Ort: {ort}. "
        f"Nutze genau eines der folgenden Verben: {', '.join(verb_sample)}. "
        f"Der Satz muss das Datum {datum_str} enthalten, allerdings nicht am Satzanfang – es soll möglichst in der Satzmitte oder am Ende erscheinen. "
        f"Gib ausschließlich den vollständigen Satz aus – keine Erklärungen, Kommentare oder Einleitungen."
    )

In [38]:
def build_prompt_type2(name, ort):
    return (
        f"Schreibe einen deutschen Satz aus der Ich-Perspektive. "
        f"In diesem Satz beschreibe ich, wie ich zusammen mit '{name}' etwas in '{ort}' tue. "
        f"Der zeitliche Kontext liegt zwischen 1819 und 1894."
        f"Verwende exakt diesen Namen: {name}, und exakt diesen Ort: {ort}. "
        f"Gib ausschließlich den vollständigen Satz aus – keine Erklärungen, Kommentare oder Einleitungen."
    )

In [39]:
# Daten laden
person_list_1 = load_list_from_file('input/person_list.txt') 
person_list_2 = load_list_from_file('input/nachname_list.txt')
ortsname_list_1 = load_list_from_file('input/ortsname_list.txt')
ortsname_list_2 = load_list_from_file("input/ortsname_SA_E_list.txt")
jahr_list = load_list_from_file("input/jahr_list.txt")
monat_list = load_list_from_file("input/monat_list.txt")
verb_list = load_list_from_file("input/verb_list.txt")

In [40]:
# Ergebnisliste vorbereiten
output_data = []

def generate_type1():
    random.shuffle(person_list_1)
    random.shuffle(ortsname_list_1)
    for name, ort in zip(person_list_1, ortsname_list_1 * ((len(person_list_1) // len(ortsname_list_1)) + 1)):
        versuche = 0
        while versuche < 6:
            monat, jahr = generate_date(raw=True)
            datum_str = f"im {monat} {jahr}"
            verb_sample = random.sample(verb_list, 10)
            prompt = build_prompt_type1(name, ort, verb_sample, datum_str)

            try:
                result = ollama.generate(model='llama3', prompt=prompt)
                satz = result['response'].strip()

                if name in satz and ort in satz and jahr in satz:
                    entities = []

                    if (span := find_span(satz, name)):
                        entities.append((*span, 'PERSON'))

                    if (span := find_span(satz, ort)):
                        entities.append((*span, 'LOC'))

                    date_phrase = f"{monat} {jahr}"
                    if (span := find_span(satz, date_phrase)):
                        entities.append((*span, 'DATE'))
                    elif (span := find_span(satz, jahr)):
                        entities.append((*span, 'DATE'))

                    output_data.append((satz, {"entities": entities}))
                    print(f"✅ [Typ1] {satz}")
                    break
                else:
                    versuche += 1
            except Exception as e:
                print(f"Fehler: {e}")
                time.sleep(1)

def generate_type2():
    random.shuffle(person_list_2)
    random.shuffle(ortsname_list_2)
    for name, ort in zip(person_list_2, ortsname_list_2 * ((len(person_list_2) // len(ortsname_list_2)) + 1)):
        versuche = 0
        while versuche < 6:
            prompt = build_prompt_type2(name, ort)
            try:
                result = ollama.generate(model='llama3', prompt=prompt)
                satz = result['response'].strip()

                if name in satz and ort in satz:
                    entities = []

                    if (span := find_span(satz, name)):
                        entities.append((*span, 'PERSON'))

                    if (span := find_span(satz, ort)):
                        entities.append((*span, 'LOC'))

                    output_data.append((satz, {"entities": entities}))
                    print(f"✅ [Typ2] {satz}")
                    break
                else:
                    versuche += 1
            except Exception as e:
                print(f"Fehler: {e}")
                time.sleep(1)


In [41]:
# Aufruf der Funktionen
generate_type1()
generate_type2()

# Speichern als JSON-kompatibles Format (repr)
with open("output/training_data.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print("\nDaten gespeichert in: training_data.json")


✅ [Typ1] Am 15. Oktober 1845 unterwegs sein muss Johann Friedrich Heinrich in Kampmann, um seine Forschungen zum geologischen Aufbau des Ortes fortzuführen, die er sich bereits im vergangenen Jahr zugelegt hat.
✅ [Typ1] Friedrich Wilhelm Kretſchmer drehen die ersten Akkorde seiner Reiseerinnerungen an Weihnachten 1827 in Bethlehem in Nord⸗America, als er sich anpassen musste an den bescheidenen Umstand, dass sein Konzertplan erst am 24. Dezember 1827 realisiert werden konnte.
✅ [Typ1] Suſanna Hauſer genoss am 20. Februar 1824 die Landschaft von Gnadenfrei, als sie sich verhalten hat, um ihre neuesten Skizzen zu entwerfen.
✅ [Typ1] Maria Roſina Doͤfner forscht im Juni 1883 in Herrnhut nach alten Handschriften, um die Geschichte der Herrnhuter Brüdergemeine besser zu verstehen und ihre Entstehung zu bestätigen.
✅ [Typ1] Am 15. Februar 1851 partizipiert Theophilus Salomon Schumann in Chriſtiansfeld aktiv an der Diskussion über die Bedeutung der Bibelinterpretation und tut dies gemeinsam m